In [1]:
# Install the Google Generative AI library and Gradio for the UI
!pip install -q -U google-generativeai gradio

In [5]:
import google.generativeai as genai
import gradio as gr
from google.colab import userdata

# --- CONFIGURATION & AUTO-DETECT ---
print("--- STARTING CONFIGURATION ---")
try:
    # 1. Load API Key
    API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=API_KEY)

    # 2. Auto-Detect Available Model
    # We ask Google: "Give us all models that support generating text"
    available_models = []
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            available_models.append(m.name)

    # 3. Select the best available model
    if not available_models:
        raise ValueError("No compatible Gemini models found for your API Key.")

    # Prefer newer models if available, otherwise take the first one
    # We look for 'flash' (fast) or 'pro' (smart)
    selected_model_name = next((m for m in available_models if 'flash' in m), available_models[0])

    model = genai.GenerativeModel(selected_model_name)
    print(f"✅ SUCCESSFULLY CONFIGURED using model: {selected_model_name}")

except Exception as e:
    print(f"❌ CONFIG ERROR: {e}")
    print("Please check your API Key and ensure 'Notebook Access' is ON in Secrets.")
    model = None

# --- CORE FUNCTIONS ---
def study_buddy_logic(input_text, mode):
    if model is None:
        return "❌ Error: API is not configured. Check the logs above."

    print(f"\n🔘 BUTTON CLICKED! Mode: {mode}")

    if not input_text.strip():
        return "⚠️ Please enter some text first!"

    # Defining prompts
    prompts = {
        "Explain Concept": f"Act as an expert tutor. Explain this in simple terms:\n\n{input_text}",
        "Summarize Notes": f"Summarize these notes into bullet points:\n\n{input_text}",
        "Generate Quiz": f"Create 3 multiple choice questions with answers at the end based on:\n\n{input_text}",
        "Create Flashcards": f"Create 5 Q&A flashcards based on:\n\n{input_text}"
    }

    # Calling AI
    try:
        print(f"⏳ Sending request to {model.model_name}...")
        response = model.generate_content(prompts[mode])
        print("✅ AI Responded!")
        return response.text
    except Exception as e:
        error_msg = f"❌ API ERROR: {str(e)}"
        print(error_msg)
        return error_msg

# --- UI SETUP ---
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("# 🎓 AI Study Buddy")

    user_input = gr.Textbox(lines=5, label="1. Paste Study Material Here")

    mode_selector = gr.Radio(
        ["Explain Concept", "Summarize Notes", "Generate Quiz", "Create Flashcards"],
        label="2. Choose Mode", value="Explain Concept"
    )

    submit_btn = gr.Button("🚀 Run AI", variant="primary")

    output_display = gr.Markdown(label="3. AI Output Result")

    submit_btn.click(fn=study_buddy_logic, inputs=[user_input, mode_selector], outputs=output_display)

print("--- LAUNCHING APP ---")
app.launch(share=True, debug=True)

--- STARTING CONFIGURATION ---
✅ SUCCESSFULLY CONFIGURED using model: models/gemini-2.5-flash


/tmp/ipython-input-3792094714.py:68: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as app:


--- LAUNCHING APP ---
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://da16792829fea9dc65.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



🔘 BUTTON CLICKED! Mode: Summarize Notes
⏳ Sending request to models/gemini-2.5-flash...
✅ AI Responded!

🔘 BUTTON CLICKED! Mode: Explain Concept
⏳ Sending request to models/gemini-2.5-flash...
✅ AI Responded!

🔘 BUTTON CLICKED! Mode: Generate Quiz
⏳ Sending request to models/gemini-2.5-flash...
✅ AI Responded!

🔘 BUTTON CLICKED! Mode: Create Flashcards
⏳ Sending request to models/gemini-2.5-flash...
✅ AI Responded!
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://da16792829fea9dc65.gradio.live
